# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os, sys
## Add current directory to sys.path to import the local "lib" folder
sys.path.append(os.path.join(os.getcwd()))

In [3]:
# TODO: Import the necessary libs
# For example: 
import json
import re
from pydantic import RootModel, BaseModel
from typing import Optional, List
from dotenv import load_dotenv
import chromadb
from lib.vector_db import VectorStore
from lib.rag import RAG
from lib.llm import LLM
from lib.state_machine import Run, Snapshot

from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from lib.evaluation import EvaluationResult, JudgeEvaluation


In [4]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [5]:
# Custom Pydantic Models for RAG and WEB search results
class GameDoc(BaseModel):
    Platform: str
    Name: str
    YearOfRelease: int
    Description: str

class GameDocList(RootModel[list[GameDoc]]):
    root: list[GameDoc]

class WebSearchResult(BaseModel):
    url: str
    title: str
    content: str
    score: float
    raw_content: Optional[str] = None

class WebSearchResultList(RootModel[list[WebSearchResult]]):
    root: list[WebSearchResult]

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [6]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
GAME_DOC_PATTERN = re.compile(
    r"""
    ^\[
        (?P<Platform>.+?)
    \]                      # closing ]
    \s+
    (?P<Name>.+?)           # game name (lazy)
    \s+
    \(
        (?P<YearOfRelease>\d{4})
    \)                      # closing )
    \s*-\s*
    (?P<Description>.+)     # rest of the line
    $""",
    re.VERBOSE | re.DOTALL,
)

def _parse_game_doc(doc: str):
    """
    Parse strings like:
    "[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer..."
    into a structured dict with keys:
    Platform, Name, YearOfRelease, Description.
    """
    m = GAME_DOC_PATTERN.match(doc.strip())
    if not m:
        # Fallback if format doesn't match
        return {
            "Platform": None,
            "Name": None,
            "YearOfRelease": None,
            "Description": doc.strip(),
        }

    data = m.groupdict()
    # Cast year to int
    data["YearOfRelease"] = int(data["YearOfRelease"])
    return data

@tool
def retrieve_game(query) -> str:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    vector_db = VectorStore(collection)
    
    results= vector_db.query(query_texts=[query])
    raw_docs = results["documents"][0] if results["documents"] else []
    parsed = [_parse_game_doc(doc) for doc in raw_docs]
    return json.dumps(parsed, ensure_ascii=False)


user_query = "What is the first 3D platformer Mario game?"
retrieved_docs = retrieve_game(user_query)
retrieved_docs

'[{"Platform": "Nintendo 64", "Name": "Super Mario 64", "YearOfRelease": 1996, "Description": "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario\'s quest to rescue Princess Peach."}, {"Platform": "Super Nintendo Entertainment System (SNES)", "Name": "Super Mario World", "YearOfRelease": 1990, "Description": "A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser."}, {"Platform": "Nintendo Switch", "Name": "Mario Kart 8 Deluxe", "YearOfRelease": 2017, "Description": "An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics."}]'

In [7]:
#user_query = "What is the first 3D platformer Mario game?"
user_query="When Pokémon Gold and Silver was released?"
retrieved_docs = retrieve_game(user_query)
retrieved_docs

'[{"Platform": "Game Boy Color", "Name": "Pokémon Gold and Silver", "YearOfRelease": 1999, "Description": "Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics."}, {"Platform": "Game Boy Advance", "Name": "Pokémon Ruby and Sapphire", "YearOfRelease": 2002, "Description": "Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles."}, {"Platform": "Nintendo 64", "Name": "Super Mario 64", "YearOfRelease": 1996, "Description": "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario\'s quest to rescue Princess Peach."}]'

#### Evaluate Retrieval Tool

In [8]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

from lib.parsers import PydanticOutputParser
from lib.evaluation import JudgeEvaluation


@tool
def evaluate_retrieval(question: str, retrieved_docs: GameDocList) -> JudgeEvaluation :
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    judge = LLM(api_key=OPENAI_API_KEY)
    judge_response = judge.invoke(
        [
            SystemMessage(
                content="""
                You are a retrieval-quality judge for a video game knowledge system.

                Your task: Evaluate whether the retrieved documents are sufficient and useful to answer the user's question accurately.

                Evaluation criteria:
                1. RELEVANCE: Does at least one document directly contain information that answers the question?
                2. SUFFICIENCY: Is the information complete enough (e.g., correct game name, platform, year) without needing external search?
                3. ACCURACY: Do the documents support a factual, unambiguous answer?

                Output requirements:
                - task_completed: True if the documents allow a confident, correct answer; False if information is missing, conflicting, or irrelevant.
                - format_correct: True if the retrieved docs are well-formed (Platform, Name, YearOfRelease, Description).
                - instructions_followed: True if the evaluation is coherent and actionable.
                - explanation: Give a concise rationale. When citing a matching document, wrap its game name in angle brackets, e.g. <Super Mario 64> or <Pokémon Gold and Silver>. If documents are insufficient, state what is missing and recommend web search."""
            ),
            UserMessage(
                content=f"Question: {question}\nRetrieved Documents: {retrieved_docs}"
            )
        ], 
        response_format=JudgeEvaluation
    )
     # Parse the structured response
    parser = PydanticOutputParser(model_class=JudgeEvaluation)
    return parser.parse(judge_response)

In [9]:
### Writy utility function to extract citation for the documents matching the
### user query from the Eval Agent explanation
import unicodedata


def _extract_and_match_citations(explanation: str, retrieved_docs: GameDocList) -> list[dict]:
    """
    Extract citations from explanation (marked by <NAME>) and match them to retrieved documents.
    
    Args:
        explanation: Text containing citations in angle brackets, e.g. "The document <Super Mario 64> answers..."
        retrieved_docs: List of dicts with keys Platform, Name, YearOfRelease, Description
        
    Returns:
        List of dicts: {"citation": str, "matched_doc": dict | None}
    """
    # Extract citations from explanation as per the eval agent instructions
    pattern = re.compile(r"<([^>]+)>")

    citations = [m.strip() for m in pattern.findall(explanation)]
    
    results = []
    if not retrieved_docs:
        return []
    try:
        docs = GameDocList.model_validate(retrieved_docs)
    except Exception as e:
        try:
            docs = GameDocList.model_validate(json.loads(retrieved_docs))
        except Exception as e:
            print(e)
            return []
    
    for cite in citations:
        matched = None
        for doc in docs.root:
            name = doc.Name.strip()
            if name and (cite.lower() == name.lower()):
                results.append({"citation": cite, "matched_doc": doc})
        
    return results

In [11]:
## Test Eval Agent
eval_result = evaluate_retrieval(user_query, retrieved_docs)
eval_result

JudgeEvaluation(task_completed=True, format_correct=True, instructions_followed=True, explanation="The document containing <Pokémon Gold and Silver> provides the correct release year of 1999, which directly answers the user's question. The information is complete and accurate, with no conflicting details.")

In [12]:
explanation = eval_result.explanation
print("Explanation: ", explanation)
citations = _extract_and_match_citations(eval_result.explanation, retrieved_docs)
print("Citations: ", citations)

Explanation:  The document containing <Pokémon Gold and Silver> provides the correct release year of 1999, which directly answers the user's question. The information is complete and accurate, with no conflicting details.
Citations:  [{'citation': 'Pokémon Gold and Silver', 'matched_doc': GameDoc(Platform='Game Boy Color', Name='Pokémon Gold and Silver', YearOfRelease=1999, Description='Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.')}]


#### Game Web Search Tool

In [13]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
from tavily import TavilyClient

@tool
def game_web_search(question)-> str:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - question: a question about game industry. 
    """
    # Write a TAVILY search query
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)    
    results= tavily_client.search(question)
    tavily_results = results["results"] if results else []
    return json.dumps(tavily_results, ensure_ascii=False)

In [75]:
def _get_web_citations(documents: str, threshold: float = 0.9) -> list[dict]:
    """
    Extract cited documents with score >= threshold, ranked from highest to lowest.

    Args:
        documents: List of dicts with at least a 'score' key.
        threshold: Minimum score (default 0.9). Documents below this are excluded.

    Returns:
        List of documents with score >= threshold, sorted by score descending.
    """
    try:
        print(">> Documents")
        # Remove the extra quotes that could make the parsing fail
        if documents.startswith('"') and documents.endswith('"'):
            documents = documents[1:-1].encode().decode('unicode_escape')  # handles \" → "
        web_docs = WebSearchResultList.model_validate(json.loads(documents))
        
    except Exception as e:
        print(e)
        return []

    return sorted(
        (d for d in web_docs.root if (d.score or 0) >= threshold),
        key=lambda d: d.score,
        reverse=True,
    )

In [76]:
web_result = game_web_search(user_query)
web_result

'[{"url": "https://en.wikipedia.org/wiki/Pok%C3%A9mon_Gold_and_Silver", "title": "Pokémon Gold and Silver - Wikipedia", "content": "***Pokémon Gold Version*** and ***Pokémon Silver Version*** are 1999 role-playing video games developed by Game Freak and published by Nintendo for the Game Boy Color. ***Pokémon Crystal Version*** is a third version after *Pokémon Gold* and *Silver*, developed by Game Freak and published by Nintendo for the Game Boy Color. *Pokémon Gold* and *Silver* were met with critical acclaim, with many saying that the extended length of gameplay and the new features were valued additions that kept the sequels as interesting as the original games. Craig Harris of *IGN* gave the games a \\"masterful\\" 10 out of 10 rating, stating that: \\"As awesome as the original Pokémon edition was, *Pokémon Gold* and *Silver* blow it away in gameplay elements, features, and goodies. *Nintendo Power* listed *Gold* and *Silver* combined as the sixth best Game Boy / Game Boy Color g

In [77]:
type(web_result)

str

In [78]:
_get_web_citations(web_result, threshold=0.75)

>> Documents


[WebSearchResult(url='https://en.wikipedia.org/wiki/Pok%C3%A9mon_Gold_and_Silver', title='Pokémon Gold and Silver - Wikipedia', content='***Pokémon Gold Version*** and ***Pokémon Silver Version*** are 1999 role-playing video games developed by Game Freak and published by Nintendo for the Game Boy Color. ***Pokémon Crystal Version*** is a third version after *Pokémon Gold* and *Silver*, developed by Game Freak and published by Nintendo for the Game Boy Color. *Pokémon Gold* and *Silver* were met with critical acclaim, with many saying that the extended length of gameplay and the new features were valued additions that kept the sequels as interesting as the original games. Craig Harris of *IGN* gave the games a "masterful" 10 out of 10 rating, stating that: "As awesome as the original Pokémon edition was, *Pokémon Gold* and *Silver* blow it away in gameplay elements, features, and goodies. *Nintendo Power* listed *Gold* and *Silver* combined as the sixth best Game Boy / Game Boy Color ga

### Agent

In [17]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
from lib.agents import Agent

games_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are a helpful agent that can answer questions about the video game industry."
        "You can use the following tools to help you answer the questions. "
        "To answer question you must following this order: \n"
        "1. Start by searching information from the vector DB"
        "2. Evaluate if the retrieved documents are enough to answer the question"
        "3. If the documents are not enough, search the web for more information"
    ),
    tools=[
        retrieve_game,
        evaluate_retrieval,
        game_web_search
    ]
)


In [18]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

results_session_1: Run = games_agent.invoke(query="When Pokémon Gold and Silver was released?", session_id="id_1")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [ ]:
def _get_snapshot_citations(snapshot: Snapshot) -> list[dict]:
    """
    Extract citations from the snapshot messages
    """
    snapshot_messages = snapshot.state_data["messages"]
    current_retrieved_docs=[]
    rag_citations= []
    web_citations= []
    for s in snapshot_messages:
        
        if not isinstance(s, ToolMessage):
            continue

        if s.name == "retrieve_game":
            # Get the list of retrieved documents
            # The retreive step should happen before the evaluation one
            current_retrieved_docs = GameDocList.model_validate_json(json.loads(s.content)).root
    
        if s.name == "evaluate_retrieval":
            # Get the valid citations based on the evaluation result
            # and last retreive results
            rag_citations = _extract_and_match_citations(s.content, current_retrieved_docs)

        if s.name == "game_web_search":
            # Get the web search results
            print(">> Web Search Result")
            web_citations = _get_web_citations(s.content, threshold=0.75)

    return rag_citations, web_citations
    
def display_citations(run: Run)-> None:
    """
    Display the citations from the last snapshot of the run
    """             
    snapshot_messages = run.snapshots[-1].state_data["messages"]
    rag_citations, web_citations = _get_snapshot_citations(run.snapshots[-1])
    print("-"*100)
    if rag_citations:
        print(">> RAG Citations:\n", rag_citations)
        print("-"*100)

    if web_citations: 
        print("@@ Web Citations:\n", web_citations)
        print("-"*100)

def invoke_agent(agent: Agent, query: str, session_id: str)-> Run:
    """
    Invoke the agent and return the run
    """

    run_results = agent.invoke(query=query, session_id=session_id) 
    print("-"*100)
    print(">> Agent answer: ", run_results.get_final_state()["messages"][-1].content)
    display_citations(run_results)
    return run_results


## Testing Agent Runs

In [ ]:
## Session 1: "When Pokémon Gold and Silver was released?"
## Session 1 (show agent short term memory): "When Pokémon Gold and Silver was released?" 
## Session 2: "Which one was the first 3D platformer Mario game?"
## Session 3: "Was Mortal Kombat X realeased for Playstation 5?"
question_1 = "When Pokémon Gold and Silver was released?"
question_2 = "Which one was the first 3D platformer Mario game?"
question_3 = "Was Mortal Kombat X realeased for Playstation 5?"

In [37]:
r1: Run = games_agent.invoke(query=question_1, session_id="id_1")
r1.get_final_state()["messages"][-1].content

r2: Run = games_agent.invoke(query=question_2, session_id="id_1")
r2.get_final_state()["messages"][-1].content

r3: Run = games_agent.invoke(query=question_3, session_id="id_3")
r3.get_final_state()["messages"][-1].content

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


'Mortal Kombat X was not specifically released for PlayStation 5. It was originally launched in 2015 for PlayStation 4, and while it can be played on PS5 through backward compatibility, there is no dedicated version for the PS5 itself.'

In [38]:
r1

Run('4074030d-9616-4565-bf57-73929165bda4')

In [ ]:
r1.snapshots

[Snapshot('1470d5d7-f7d2-4c1d-8d3c-71568e69a0a8') @ [2026-02-15 18:18:11.448188]: __entry__.State({'user_query': 'When Pokémon Gold and Silver was released?', 'instructions': 'You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for more information', 'messages': [SystemMessage(role='system', content='You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for 

In [41]:
rX: Run = games_agent.invoke(query=question_1, session_id="id_X")



[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [42]:
rX.snapshots

[Snapshot('932f0f1c-d392-4f60-ab24-b629efdc288b') @ [2026-02-15 18:20:00.112518]: __entry__.State({'user_query': 'When Pokémon Gold and Silver was released?', 'instructions': 'You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for more information', 'messages': [], 'current_tool_calls': None, 'session_id': 'id_X'}),
 Snapshot('82258399-a6fd-4a20-8171-fc38345d7819') @ [2026-02-15 18:20:00.112644]: message_prep.State({'user_query': 'When Pokémon Gold and Silver was released?', 'instructions': 'You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must followi

### Session 1: "When Pokémon Gold and Silver was released?"

In [114]:
r45 = invoke_agent(games_agent, "When Pokémon Gold and Silver was released?", "id_45")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
----------------------------------------------------------------------------------------------------
>> Agent answer:  Pokémon Gold and Silver was released in 1999 for the Game Boy Color.
----------------------------------------------------------------------------------------------------
>> RAG Citations:
 [{'citation': 'Super Mario 64', 'matched_doc': GameDoc(Platform='Nintendo 64', Name='Super Mario 64', YearOfRelease=1996, Description="A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.")}]
----------------------------------------------------------------------------------------------------


In [115]:
r45.get_final_state()

{'user_query': 'When Pokémon Gold and Silver was released?',
 'instructions': 'You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for more information',
 'messages': [SystemMessage(role='system', content='You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for more information'),
  UserMessage(role='user', content='When Pokémon Gold and Silver was release

In [45]:
## When sending the same reauest in the same session
## We can see that the sequence of tool calls is shorter (only 2 tool calls)
## This is because the agent is able to reuse the context from the previous run
## and did not require to run the RAG and Evaluation tool again
## This is a good example showing that agent remembers the context from the previous run
invoke_agent(games_agent, "When Pokémon Gold and Silver was released?", "id_1")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
----------------------------------------------------------------------------------------------------
>> Agent answer:  Pokémon Gold and Silver was released in 1999 for the Game Boy Color.
----------------------------------------------------------------------------------------------------
>> RAG Citations:
 [{'citation': 'Super Mario 64', 'matched_doc': GameDoc(Platform='Nintendo 64', Name='Super Mario 64', YearOfRelease=1996, Description="A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.")}]
----------------------------------------------------------------------------------------------------


Run('32202f45-d0d7-40da-a33c-c691f07a217f')

In [82]:
invoke_agent(games_agent, "Which one was the first 3D platformer Mario game?", "id_45")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
----------------------------------------------------------------------------------------------------
>> Agent answer:  The first 3D platformer Mario game is **Super Mario 64**, which was released in 1996 for the Nintendo 64.
----------------------------------------------------------------------------------------------------
>> RAG Citations:
 [{'citation': 'Super Mario 64', 'matched_doc': GameDoc(Platform='Nintendo 64', Name='Super Mario 64', YearOfRelease=1996, Description="A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.")}]
----------------------------------------------------------------------------------------------------


Run('d6423619-550a-4aa7-8844-32f01889e9e5')

In [118]:
invoke_agent(games_agent, "Was Mortal Kombat X realeased for Playstation 5?", "id_3")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
----------------------------------------------------------------------------------------------------
>> Agent answer:  No, Mortal Kombat X was not specifically released for PlayStation 5. It was originally launched for PlayStation 4 in 2015. However, you can play Mortal Kombat X on a PS5 through backward compatibility if you have the PS4 version.


TypeError: 'Run' object is not subscriptable

In [58]:
games_agent.get_session_runs(session_id="id_3")[-1].snapshots

[Snapshot('98107af2-f9bf-4b32-8a39-0bec2efee08a') @ [2026-02-15 18:22:43.239326]: __entry__.State({'user_query': 'Was Mortal Kombat X realeased for Playstation 5?', 'instructions': 'You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for more information', 'messages': [SystemMessage(role='system', content='You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the we

In [51]:
games_agent.get_session_runs(session_id="id_3")[-1].snapshots[-1]

Snapshot('a3242522-a76d-4a8e-81ef-ceb463872719') @ [2026-02-15 18:22:45.127450]: llm_processor.State({'user_query': 'Was Mortal Kombat X realeased for Playstation 5?', 'instructions': 'You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the web for more information', 'messages': [SystemMessage(role='system', content='You are a helpful agent that can answer questions about the video game industry.You can use the following tools to help you answer the questions. To answer question you must following this order: \n1. Start by searching information from the vector DB2. Evaluate if the retrieved documents are enough to answer the question3. If the documents are not enough, search the

### (Optional) Advanced

In [70]:
TEST = "\"[{\"url\": \"https://www.youtube.com/watch?v=HUNMZwDdrvY\", \"title\": \"Mortal Kombat X (PS5) 4K 60FPS HDR Gameplay - YouTube\", \"content\": \"Mortal Kombat X (PS5) 4K 60FPS HDR Gameplay Mortal Kombat X[b] is a 2015 fighting game developed by NetherRealm Studios and published by\", \"score\": 0.99959236, \"raw_content\": null}, {\"url\": \"https://en.wikipedia.org/wiki/Mortal_Kombat_X\", \"title\": \"Mortal Kombat X - Wikipedia\", \"content\": \"***Mortal Kombat X*** is a 2015 fighting game developed by NetherRealm Studios and published by Warner Bros. An upgraded version of *Mortal Kombat X*, titled ***Mortal Kombat XL***, was released on March 1, 2016, for PlayStation 4 and Xbox One, including all downloadable content characters from the two released Kombat Packs, almost all bonus alternate costumes available at the time of release, improved gameplay, and improved netcode. By July 2015, due to heavy criticism for the porting issues that plagued the PC release of the game, almost all references to *Mortal Kombat X* had been removed from High Voltage Software's Facebook page. On March 2, 2015, NetherRealm Studios announced that their mobile division would release an iOS/Android \\\"Android (operating system)\\\") version of *Mortal Kombat X* in April 2015. With the 1.11 update version of the mobile game released on December 6, 2016, Freddy Krueger who appeared as a DLC character in *MK9 \\\"Mortal Kombat (2011 video game)\\\")* was added as a mobile-exclusive character using his signature moves and X-Ray attack from MK9.\", \"score\": 0.9992679, \"raw_content\": null}, {\"url\": \"https://www.playstation.com/en-us/games/mortal-kombat-x_msm_moved/\", \"title\": \"Mortal Kombat X - PlayStation\", \"content\": \"* Supports up to 10 online players with PS Plus. ## Ratings and reviews. Every review comes from a verified owner of this game or item and is evaluated by a team of moderators. Check the Ratings and Reviews Policy for more details. ### Report Review. ### Report Review. Your report will be reviewed by PlayStation Safety, and action will be taken if appropriate. Only owners of this game can rate it. ### No ratings and reviews. Be the first to add a rating and review. ### Your review. ### Thank you for submitting your review. It may take up to 72 hours for your review to be posted. To play this game on PS5, your system may need to be updated to the latest system software. 'MORTAL KOMBAT X software © 2015 Warner Bros. WB GAMES LOGO, WB SHIELD, NETHERREALM LOGO, MORTAL KOMBAT, THE DRAGON LOGO, and all related characters and elements are trademarks of and © Warner Bros.\", \"score\": 0.9988927, \"raw_content\": null}, {\"url\": \"https://store.playstation.com/en-us/product/UP1018-CUSA00967_00-MORTALKOMBATX000\", \"title\": \"Mortal Kombat X - PlayStation Store\", \"content\": \"# Mortal Kombat X. ### Mortal Kombat X. * Mortal Kombat X XL Pack. * Mortal Kombat X Unlock All Krypt Items. * Mortal Kombat X Jason Voorhees. * Mortal Kombat X Kombat Pack 2. * Mortal Kombat X Tanya Bundle. * Mortal Kombat X Predator Bundle. * Mortal Kombat X Tremor Bundle. * Mortal Kombat X Apocalypse Pack. * Mortal Kombat X Samurai Pack. * Mortal Kombat X Horror Pack. * Mortal Kombat X Kold War Pack. * Mortal Kombat X Klassic Pack 1. * Mortal Kombat X Brazil Pack. * Mortal Kombat X Predator/Prey Pack. * Mortal Kombat X Klassic Pack 2. * Mortal Kombat X Cosplay Pack. ## Ratings and reviews. Check the Ratings and Reviews Policy for more details. ### Mortal Kombat X. 'MORTAL KOMBAT X software © 2015 Warner Bros. WB GAMES LOGO, WB SHIELD, NETHERREALM LOGO, MORTAL KOMBAT, THE DRAGON LOGO, and all related characters and elements are trademarks of and © Warner Bros.\", \"score\": 0.99845123, \"raw_content\": null}, {\"url\": \"https://www.youtube.com/watch?v=tqsw711ZuAk\", \"title\": \"Mortal Kombat X - PS5 Gameplay - YouTube\", \"content\": \"Mortal Kombat X - PS5 Gameplay\\nSection Plays\\n194000 subscribers\\n102 likes\\n12696 views\\n4 Jun 2025\\nMortal Kombat X - PS5 Gameplay\\n\\nAbout this game :\\nMortal Kombat X is a 2015 fighting game developed by NetherRealm Studios and published by Warner Bros. Interactive Entertainment for Microsoft Windows, PlayStation 4, and Xbox One. It is the tenth main installment in the Mortal Kombat series and a sequel to Mortal Kombat (2011), taking place 25 years later after the events of its predecessor. High Voltage Software developed the Windows version of the game, with Polish studio QLOC taking over the work on it shortly after the release of Kombat Pack 1.\\n\\nWho’s Next? Experience the Next Generation of the 1 Fighting Franchise.\\n\\nMortal Kombat X combines unparalleled, cinematic presentation with all new gameplay. For the first time, players can choose from multiple variations of each character impacting both strategy and fighting style.\\n\\n#ps5gameplay\\n9 comments\\n\", \"score\": 0.9977482, \"raw_content\": null}]\""
TEST

'"[{"url": "https://www.youtube.com/watch?v=HUNMZwDdrvY", "title": "Mortal Kombat X (PS5) 4K 60FPS HDR Gameplay - YouTube", "content": "Mortal Kombat X (PS5) 4K 60FPS HDR Gameplay Mortal Kombat X[b] is a 2015 fighting game developed by NetherRealm Studios and published by", "score": 0.99959236, "raw_content": null}, {"url": "https://en.wikipedia.org/wiki/Mortal_Kombat_X", "title": "Mortal Kombat X - Wikipedia", "content": "***Mortal Kombat X*** is a 2015 fighting game developed by NetherRealm Studios and published by Warner Bros. An upgraded version of *Mortal Kombat X*, titled ***Mortal Kombat XL***, was released on March 1, 2016, for PlayStation 4 and Xbox One, including all downloadable content characters from the two released Kombat Packs, almost all bonus alternate costumes available at the time of release, improved gameplay, and improved netcode. By July 2015, due to heavy criticism for the porting issues that plagued the PC release of the game, almost all references to *Mortal K

In [72]:
WebSearchResultList.model_validate(TEST)

ValidationError: 1 validation error for WebSearchResultList
  Input should be a valid list [type=list_type, input_value='"[{"url": "https://www.y... "raw_content": null}]"', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/list_type